In [1]:
import torch
import torch.nn as nn
from jcopdl.callback import Callback, set_config
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\Toko Ferdi\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset & DataLoader

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
# kalo bentukanya data tabular, tinggal ganti image folder jadi tensor dataset
# train_set = datasets.TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
# kalo bentuknya speech, tinggal ganti image folder jadi audio folder
# train_set = datasets.AudioFolder("../../data neural network/train", transform=transforms.ToTensor())

transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(),
    # transforms.Normalize((0.1307,), (0.3081,))
])

# kalo bentuknya image, tinggal ganti tensor dataset jadi image folder
train_set = datasets.ImageFolder("../../data neural network/train", transform=transform)
test_set = datasets.ImageFolder("../../data neural network/test", transform=transform)

train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=4)

In [4]:
images, labels = next(iter(train_loader))

In [5]:
# MCHW (batch / jumlah gambar, channel, height, width)
images.shape, labels.shape

(torch.Size([128, 1, 28, 28]), torch.Size([128]))

In [6]:
train_set.class_to_idx

{'Ankle boot': 0,
 'Bag': 1,
 'Coat': 2,
 'Dress': 3,
 'Pullover': 4,
 'Sandal': 5,
 'Shirt': 6,
 'Sneaker': 7,
 'T-shirt or Top': 8,
 'Trouser': 9}

In [7]:
label2cat = train_set.classes

In [8]:
label2cat

['Ankle boot',
 'Bag',
 'Coat',
 'Dress',
 'Pullover',
 'Sandal',
 'Shirt',
 'Sneaker',
 'T-shirt or Top',
 'Trouser']

# Arsitektur & Config

In [9]:
from jcopdl.layers import linear_block

In [10]:
class fashionClf(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Flatten(),
            linear_block(784, 512, dropout=0.2, batch_norm=True),
            linear_block(512, 256, dropout=0.2, batch_norm=True),
            linear_block(256, 128, dropout=0.2, batch_norm=True),
            linear_block(128, 64, dropout=0.2, batch_norm=True),
            linear_block(64, 10, activation="lsoftmax", batch_norm=False)
        )
        
    def forward(self, x):
        return self.fc(x)

model = fashionClf().to(device)
model

fashionClf(
  (fc): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Sequential(
      (0): Linear(in_features=784, out_features=512, bias=True)
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.2, inplace=False)
    )
    (2): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.2, inplace=False)
    )
    (3): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.2, inplace=False)
    )
    (4): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (

In [11]:
config = set_config({
  "input_size":784, 
  "n1":256, 
  "n2":64, 
  "output_size":len(train_set.classes),
  "dropout":0.2
})

In [12]:
class fashionClassifier(nn.Module):
    def __init__(self, input_size, n1, n2, output_size, dropout=0.2):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Flatten(),
            linear_block(input_size, n1, dropout=dropout, batch_norm=True),
            linear_block(n1, n2, dropout=0.2, batch_norm=True),
            linear_block(n2, output_size, activation="lsoftmax", batch_norm=False)
        )
        
    def forward(self, x):
        return self.fc(x)

# cari set config yg bukan dari wira

model = fashionClassifier(input_size=784, n1=256, n2=64, output_size=len(train_set.classes)).to(device)

# Training Preparation => MCOC

In [13]:
model = fashionClassifier(input_size=784, n1=256, n2=64, output_size=len(train_set.classes)).to(device)
criterion = nn.NLLLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
callback = Callback(model, config, outdir="model")

# Training

In [14]:
from tqdm.auto import tqdm

def loop(mode, datasets, dataloader, model, criterion, optimizer, device):
    if mode == "train":
        model.train()
    else:
        model.eval()
    cost = correct = 0
    for feature, target in dataloader:
        feature, target = feature.to(device), target.to(device)
        output = model(feature)
        loss = criterion(output, target)
        
        if mode == "train":
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        cost += loss.item() * len(feature)
        correct += (output.argmax(dim=1) == target).sum().item()
    cost /= len(datasets)
    acc = correct / len(datasets)
    return cost, acc


In [15]:
# def train(model, criterion, optimizer, train_loader, test_loader, device, epochs=10):
#     for epoch in range(1, epochs+1):
#         train_cost, train_acc = loop("train", train_loader.dataset, train_loader, model, criterion, optimizer, device)
#         test_cost, test_acc = loop("test", test_loader.dataset, test_loader, model, criterion, optimizer, device)
#         print(f"Epoch {epoch:2d} | train_cost: {train_cost:.4f} | train_acc: {train_acc:.4f} | test_cost: {test_cost:.4f} | test_acc: {test_acc:.4f}")
    
# train(model, criterion, optimizer, train_loader, test_loader, device, epochs=10)

In [17]:
while True:
  train_cost, train_score = loop("train", train_loader.dataset, train_loader, model, criterion, optimizer, device)
  with torch.no_grad():
    test_cost, test_score = loop("test", test_loader.dataset, test_loader, model, criterion, optimizer, device)

  # logging
  callback.log(train_cost, train_score, test_cost, test_score)

  # checkpoint
  callback.save_checkpoint()

  # runtime plot
  callback.cost_runtime_plotting()
  callback.score_runtime_plotting()

  if callback.early_stopping(model, monitor = "test_score"):
    callback.plot_cost()
    callback.plot_score()
    print("Early stopping")
    break
  



Epoch     1
Train_cost  = 0.4224 | Test_cost  = 0.8491 | Train_score = 0.3679 | Test_score = 0.8645 |

Epoch     2
Train_cost  = 0.3839 | Test_cost  = 0.8625 | Train_score = 0.3661 | Test_score = 0.8598 |
==> EarlyStop patience =  1 | Best test_score: 0.8645

Epoch     3
Train_cost  = 0.3537 | Test_cost  = 0.8706 | Train_score = 0.3466 | Test_score = 0.8706 |

Epoch     4
Train_cost  = 0.3378 | Test_cost  = 0.8756 | Train_score = 0.3405 | Test_score = 0.8728 |

Epoch     5
Train_cost  = 0.3246 | Test_cost  = 0.8800 | Train_score = 0.3658 | Test_score = 0.8605 |
==> EarlyStop patience =  1 | Best test_score: 0.8728

Epoch     6
Train_cost  = 0.3167 | Test_cost  = 0.8836 | Train_score = 0.3307 | Test_score = 0.8763 |

Epoch     7
Train_cost  = 0.3023 | Test_cost  = 0.8889 | Train_score = 0.3398 | Test_score = 0.8690 |
==> EarlyStop patience =  1 | Best test_score: 0.8763

Epoch     8
Train_cost  = 0.2918 | Test_cost  = 0.8923 | Train_score = 0.3361 | Test_score = 0.8766 |

Epoch     9
T